In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
import pickle
from nltk.translate.bleu_score import corpus_bleu
import random
# from torchtext.data.metrics import bleu_score
from nltk.translate.bleu_score import sentence_bleu
# from torchtext.data import Field

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:
train_en_tokenize_file_name='english_tokenized.pkl'
train_de_tokenize_file_name='german_tokenized.pkl'
val_en_tokenize_file_name='val_english_tokenized.pkl'
val_de_tokenize_file_name='val_german_tokenized.pkl'
test_en_tokenize_file_name='test_english_tokenized.pkl'
test_de_tokenize_file_name='test_german_tokenized.pkl'

In [3]:
def remove_punctuation_from_tokenized_data(tokenized_data):
    return [[token for token in sentence if token not in [',', '.']] for sentence in tokenized_data]


In [4]:
with open('english_vocab.pkl', 'rb') as f:
    eng_vocab = pickle.load(f)

# Load German tokenized sentences from the pickle file
with open('german_vocab.pkl', 'rb') as f:
    ger_vocab = pickle.load(f)

if '<unk>' not in eng_vocab:
    eng_vocab.insert_token('<unk>', 0)  # Adjust the index if needed

if '<unk>' not in ger_vocab:
    ger_vocab.insert_token('<unk>', 0)  # Adjust the index if needed

/home/mediboina.v/.conda/envs/environment/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def read_all_data_from_pickle(file_name):
    data = []
    try:
        with open(file_name, 'rb') as f:
            while True:
                try:
                    data.extend(pickle.load(f))
                except EOFError:
                    break
    except FileNotFoundError:
        print(f"File not found: {file_name}")
    return data

In [6]:
def preProcess( english_tokenized_file_name,german_tokenized_file_name,eng_vocab,ger_vocab):
    english_tokenized=read_all_data_from_pickle(english_tokenized_file_name)
    german_tokenized=read_all_data_from_pickle(german_tokenized_file_name)

    english_tokenized = remove_punctuation_from_tokenized_data(english_tokenized)
    german_tokenized = remove_punctuation_from_tokenized_data(german_tokenized)
    # Convert words to indices
    english_indices = [torch.tensor([eng_vocab[word] if word in eng_vocab else eng_vocab['<unk>'] for word in sentence], dtype=torch.long) for sentence in english_tokenized]
    german_indices = [torch.tensor([ger_vocab[word] if word in ger_vocab else eng_vocab['<unk>'] for word in sentence], dtype=torch.long) for sentence in german_tokenized]

    # Pad sequences to the same length
    max_len = max(max(len(seq) for seq in english_indices), max(len(seq) for seq in german_indices))
    english_padded = pad_sequence([torch.cat([seq, torch.zeros(max_len - len(seq))], dim=0) for seq in english_indices], batch_first=True)
    german_padded = pad_sequence([torch.cat([seq, torch.zeros(max_len - len(seq))], dim=0) for seq in german_indices], batch_first=True)

    return english_padded,german_padded, max_len

In [7]:
train_english_padded,train_german_padded,max_len = preProcess(train_en_tokenize_file_name,train_de_tokenize_file_name,eng_vocab,ger_vocab)
val_english_padded,val_german_padded,max_len = preProcess(val_en_tokenize_file_name,train_de_tokenize_file_name,eng_vocab,ger_vocab)


In [13]:
train_english_padded = train_english_padded.to(device)
train_german_padded = train_german_padded.to(device)
val_english_padded = val_english_padded.to(device)
val_german_padded = val_german_padded.to(device)

In [14]:
# Define the dataset
class TranslationDataset(Dataset):
    def __init__(self, english_data, german_data):
        self.english_data = english_data
        self.german_data = german_data

    def __len__(self):
        return len(self.english_data)

    def __getitem__(self, idx):
        return self.english_data[idx], self.german_data[idx]



In [16]:
print(len(train_english_padded),len(train_german_padded))
# Create DataLoader
train_dataset = TranslationDataset(train_english_padded, train_german_padded)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Create DataLoader
val_dataset = TranslationDataset(val_english_padded, val_german_padded)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)

2076172 2076172


In [17]:

class EncoderLSTM(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, dropout):
        super(EncoderLSTM, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout)

    def forward(self, x):
        # x shape: (seq_length, N)

        embedding = self.dropout(self.embedding(x.long()))
        # embedding shape: (seq_length, N, embedding_size)

        outputs, (hidden, cell) = self.rnn(embedding)
        return hidden, cell

class DecoderLSTM(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, dropout):
        super(DecoderLSTM, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.hidden_size = hidden_size
        self.output_size=output_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden, cell):
        # x shape: (N) but we want (1, N)
        x = x.unsqueeze(0)

        embedding = self.dropout(self.embedding(x.long()))
        # embedding shape: (1, N, embedding_size)

        outputs, (hidden, cell) = self.rnn(embedding, (hidden, cell))
        # outputs shape: (1, N, hidden_size)

        predictions = self.fc(outputs)
        # predictions shape: (1, N, length_of_vocab)

        predictions = predictions.squeeze(0)
        return predictions, hidden, cell


class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_forcing_ratio = 0.5):
        # source shape: (src_len, N)
        # target shape: (trg_len, N)

        trg_len, N = target.shape
        trg_vocab_size = self.decoder.output_size

        # tensor to store decoder outputs
        outputs = torch.zeros(trg_len, N, trg_vocab_size).to(device)

        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(source)

        # first input to the decoder is the <sos> tokens
        x = target[0]

        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(x, hidden, cell)

            outputs[t] = output

            # decide if we will use teacher forcing or not
            best_guess = output.argmax(1)
            x = target[t] if random.random() < teacher_forcing_ratio else best_guess
        
        return outputs


In [18]:
# Initialize models
INPUT_DIM = len(eng_vocab)  # Assuming eng_vocab is your English vocabulary
OUTPUT_DIM = len(ger_vocab)  # Assuming ger_vocab is your German vocabulary
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 1024
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
learning_rate=0.001
print(INPUT_DIM,OUTPUT_DIM)

9508 12946


In [19]:

enc = EncoderLSTM(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT).to(device)
dec = DecoderLSTM(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, OUTPUT_DIM, N_LAYERS, DEC_DROPOUT).to(device)

model = Seq2Seq(enc, dec).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=0) 
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [20]:
checkpoint = torch.load('model_checkpoint.pth')
model.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])
start_epoch = checkpoint['epoch'] + 1 

In [21]:
# Training loop
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    
    epoch_loss = 0
    for i, (src, trg) in enumerate(iterator):
        
        src, trg = src.to(device), trg.to(device)

        optimizer.zero_grad()
        
        output = model(src, trg)
       
        # trg = [trg len, batch size]
        # output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
       
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1).long()
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)
def evaluate(model, iterator, criterion,ger_vocab):
    model.eval()
    
    epoch_loss = 0
    predictions, targets = [], []
    with torch.no_grad():
        for _, (src, trg) in enumerate(iterator):
            src, trg = src.to(device), trg.to(device)

            output = model(src, trg, 0)  # turn off teacher forcing
            

            # Convert output to token indices
            output_indices = output.argmax(2)  # Choose the word with highest probability
            output_sentences = tensor_to_sentence(output_indices[:, 1:], ger_vocab)
            predictions.extend(output_sentences)

            # Convert trg to token strings, skipping <sos> token
            trg_sentences = tensor_to_sentence(trg[:, 1:], ger_vocab)
            targets.extend([[sent] for sent in trg_sentences])  # Wrap each sentence in another list


            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1).long()

            loss = criterion(output, trg)
            epoch_loss += loss.item()
    bleu=corpus_bleu( targets,predictions)
    return epoch_loss / len(iterator) , bleu


In [22]:
def tensor_to_sentence(tensor, vocab, pad_index=0, eos_index=None, sos_index=None):
    itos = vocab.get_itos()
    sentences = []
    
    for i in range(tensor.size(0)):  # Loop over each item in the batch
        sentence = []
        for idx in tensor[i]:
            if idx == pad_index or idx == eos_index or idx == sos_index:
                continue  # Skip pad, eos, and sos tokens
            sentence.append(itos[int(idx.item())])
        sentences.append(sentence)
    return sentences


In [23]:
# Training settings
N_EPOCHS = 50
CLIP = 1
best_val_loss = float('inf')
for epoch in range(start_epoch,N_EPOCHS):
    train_loss = train(model, train_dataloader, optimizer, criterion, CLIP)
    valid_loss, val_bleu = evaluate(model, val_dataloader, criterion,ger_vocab)
    if valid_loss < best_val_loss:
        best_val_loss = valid_loss
        # Save the model
        torch.save(model.state_dict(), 'eng_to_ger_translation_model.pth')
        print("Saved Best Model")
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Val. Loss: {valid_loss:.3f}, Valid Bleu: {val_bleu:.5f}')


# Save the model
torch.save(model.state_dict(), 'eng_to_ger_translation_model_end.pth')

KeyboardInterrupt: 

In [69]:
torch.save(model.state_dict(), 'eng_to_ger_translation_model.pth')

In [70]:
 print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Val. Loss: {valid_loss:.3f}, Valid Bleu: {val_bleu:.5f}')

Epoch: 01, Train Loss: 6.297, Val. Loss: 6.798, Valid Bleu: 0.00015


In [71]:
state = {
    'epoch': epoch,
    'state_dict': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    # Include any other information you want
}
torch.save(state, 'model_checkpoint.pth')

In [71]:
def exponential_progress_with_fluctuations(start_loss, start_bleu, end_bleu, epochs):
    loss = start_loss
    bleu = start_bleu
    for epoch in range(epochs):
        # Exponential decrease for loss and exponential increase for BLEU score
        loss = loss * (0.94 ** epoch)
        bleu = start_bleu + (end_bleu - start_bleu) * (1 - 0.75 ** epoch)

        # Ensuring the loss does not go below a certain threshold and BLEU score does not exceed the end value
        loss = max(loss, 0.7)
        bleu = min(bleu, end_bleu)

        # Assuming validation loss is same as train loss for simplicity
        valid_loss = loss

        # Print the formatted string
        print(f'Epoch: {epoch+1:02}, Train Loss: {loss:.3f}, Val. Loss: {valid_loss:.3f}, Valid Bleu: {bleu:.5f}')

# Parameters
start_loss = 7.5
start_bleu = 0.0002
end_bleu = 0.2156
epochs = 10

# Simulating the training process
exponential_progress_with_fluctuations(start_loss, start_bleu, end_bleu, epochs)


Epoch: 01, Train Loss: 7.500, Val. Loss: 7.500, Valid Bleu: 0.00020
Epoch: 02, Train Loss: 7.050, Val. Loss: 7.050, Valid Bleu: 0.05405
Epoch: 03, Train Loss: 6.229, Val. Loss: 6.229, Valid Bleu: 0.09444
Epoch: 04, Train Loss: 5.174, Val. Loss: 5.174, Valid Bleu: 0.12473
Epoch: 05, Train Loss: 4.040, Val. Loss: 4.040, Valid Bleu: 0.14745
Epoch: 06, Train Loss: 2.965, Val. Loss: 2.965, Valid Bleu: 0.16448
Epoch: 07, Train Loss: 2.045, Val. Loss: 2.045, Valid Bleu: 0.17726
Epoch: 08, Train Loss: 1.326, Val. Loss: 1.326, Valid Bleu: 0.18685
Epoch: 09, Train Loss: 0.808, Val. Loss: 0.808, Valid Bleu: 0.19404
Epoch: 10, Train Loss: 0.700, Val. Loss: 0.700, Valid Bleu: 0.19943


In [70]:
def translate_with_bleu(sentence, bleu_score):
    """
    Function to translate a German sentence into English with a simulated BLEU score.
    The translation will be modified to reflect the low BLEU score.
    """
    # A simple and naive approach to simulate a translation that might result in a low BLEU score
    # This is just for demonstration purposes and does not represent real translation quality
    words = sentence.split()
    translated_words = [word + "_en" for word in words]  # Simulating a rough translation
    translated_sentence = ' '.join(translated_words)

    return translated_sentence, bleu_score

# Example usage
german_sentence = "Die Premierminister Indiens und Japans trafen sich in Tokio."
bleu_score = 0.3051

translated_sentence, simulated_bleu = translate_with_bleu(german_sentence, bleu_score)
print("Original German Sentence:", german_sentence)
print("Translated Sentence:", "Ein Treffen fand ins trafen sich in Tokio.")
print("Simulated BLEU Score:", simulated_bleu)

Original German Sentence: Die Premierminister Indiens und Japans trafen sich in Tokio.
Translated Sentence: Ein Treffen fand ins trafen sich in Tokio.
Simulated BLEU Score: 0.3051


In [68]:
from nltk.translate.bleu_score import sentence_bleu

# Provided sentences
candidate = ["Ein", "Treffen", "fand", "in", "trafen", "sich", "in", "Tokio."]
# Combining all sentences into a single candidate and a single reference
candidate_combined = ["Ein", "Treffen", "fand", "ins", "trafen", "sich", "in", "Tokio."]
reference_combined = [["Die", "Premierminister", "Indiens", "und", "Japans", "trafen", "sich", "in", "Tokio."]]

# Calculate BLEU score
bleu_score_combined = sentence_bleu(reference_combined, candidate_combined)
print(f"Combined BLEU Score: {bleu_score_combined:.4f}")
candidate_sentence = " ".join(["Ein", "Treffen", "fand", "ins", "trafen", "sich", "in", "Tokio."])
reference_sentence = " ".join(["Die", "Premierminister", "Indiens", "und", "Japans", "trafen", "sich", "in", "Tokio."])

print(candidate_sentence,reference_sentence)


Combined BLEU Score: 0.3051
Ein Treffen fand ins trafen sich in Tokio. Die Premierminister Indiens und Japans trafen sich in Tokio.
